In [4]:
import pandas as pd
import altair as alt
import lib.utils.viz.KD as KD
folder="./Cifar10/ResNet101/normal"

In [5]:
import altair as alt
from lib.utils.viz import Feat


In [6]:
data=Feat.load_data(folder)

In [7]:
data.head()

,Unnamed: 0,test_acc,test_teacher/acc,test_loss,test_eval,train_acc,train_teacher/acc,train_loss,train_eval,epoch,...,teacher,distillation,last_layer,student_layer,teacher_layer,layer,transform,dataset,exp_name,student last_layer
0,0,88.77,93.94,0.015080,1.395208,94.672,99.886,0.009206,0.413442,50,...,ResNet101,att_max,"KD,T-8",6,26,0,"none,","cifar10,",las,"MobileNet,KD,T-8"
1,1,89.10,93.94,0.014359,1.279313,94.698,99.884,0.009395,0.416910,50,...,ResNet101,att_max,"KD,T-8",15,56,1,"none,","cifar10,",las,"MobileNet,KD,T-8"
2,2,89.04,93.94,0.015527,1.352884,94.832,99.908,0.009360,0.395154,50,...,ResNet101,att_max,"KD,T-8",26,219,2,"none,","cifar10,",las,"MobileNet,KD,T-8"
3,3,89.46,93.94,0.019441,1.298690,95.106,99.906,0.014023,0.385567,50,...,ResNet101,att_max,"KD,T-8",55,239,3,"none,","cifar10,",las,"MobileNet,KD,T-8"
4,4,89.50,93.94,0.015188,1.347961,94.724,99.880,0.009325,0.408336,50,...,ResNet101,att_mean,"KD,T-8",6,26,0,"none,","cifar10,",las,"MobileNet,KD,T-8"


In [12]:
detalle=['test_acc', 'test_loss', 'test_eval',
               'train_acc','train_loss', 'train_eval']


scale="log"
chart=alt.Chart(data).mark_point().encode(
    alt.X(alt.repeat("column"), 
          type='quantitative',
          scale=alt.Scale(zero=False,base=10,type=scale)),
    alt.Y(alt.repeat("row"), 
          type='quantitative',
          scale=alt.Scale(zero=False,base=10,type=scale)),
    color='student last_layer',
    shape='distillation'
).properties(
    width=200,
    height=200
).repeat(
    row=detalle,
    column=detalle
)

chart

alt.RepeatChart(...)

# Experimentos
En general se quiere probar que
1- Los algoritmos permiten aprender

2- Los algoritmos generalizan

3- Los algoritmos entrenan mas rápido que la wea cruda (Estabilidad, velocidad)

5- Cuanto depende del dataset ese aprendizaje

4- Son capaces de aprender de alguna distribución generada en la vecindad del dataset

6- Permiten una mejor estabilidad frente al ruido que el entrenamiento en crudo (En que momento pasa el umbral de tanto x)

7- Feats permiten una mejor estabilidad frente al ruido que el uso de logits solo

8- Funcionamiento de feats > funcionamiento de KD > funcionamiento de CE solo


Por cada experimento interesa

1- velocidad de convergencia (estabilidad?) (En que epoca el algoritmo pasa 3 epochs sin crecer)

2- Acc en test 

3- Acc en test / train

4- Mejores y peores

5- Delta sobre KD, CE

6- grafico que permita ver tendencia según tecnica/capa

7- media de feats por capa

Los experimentos disponibles sobre Cifar 10 son

| Experimento    | ce | kd | feat |   |
|----------------|----|----|------|---|
| Dataset Normal | x  | x  | x    |   |
| Ruido  (0,1-1) |    | x  | x    |   |
| GAN            |    | x  | x    |   |
| VAE            |    | x  | x    |   |

Podria faltar grid de ruido con temperatura
Podría ser interesante probar con alguna medida de distancia entre y_t y y_s, implica correr todo de nuevo

### Carga de datos

In [ ]:
import pandas as pd
import altair as alt


CE_path=["Cifar10/%s/record.json"%model for model in ["ResNet101","ResNet18","MobileNet"]]

import json
def load_teachers(CE_path):
    def flatten_record(path):
        i=json.load(open(path,'r'))
        d={}
        for k,v in i.items():
            if isinstance(v, dict) :
                for l in v.values():
                    for ki,vi in l.items():
                        #print(l.values())
                        d["/".join([k,ki])]=vi
            else:
                d[k]=v
        return d

    d={'model':[],
     'test/acc': [],
     'test/loss ': [],
     'train/acc': [],
     'train/loss ': [],
     'epoch': [],
     'train_step': [],
     'test_step': []}

    for path in CE_path:
        d['model'].append(path.split("/")[-2])
        for k,v in flatten_record(path).items():
            d[k].append(v)
    return pd.DataFrame.from_dict(d)

data={}
data['teachers']=load_teachers(CE_path)
data["students"]={'feat':{},
                  'kd':{}}


import glob
paths=glob.glob("Cifar10/**/summary.csv",recursive=True)
for path in paths:
    ks=path.split("/")[-2].split("_")
    data["students"][ks[0]][ks[1]]=pd.read_csv(path)



In [ ]:
data['students']['kd']['normal']

In [ ]:
source=data['students']['kd']['normal'].copy()
source["test_acc"]

### Graficos Dataset Normal
Tabla teachers, (mejores, peores, media) en KD y con feats

1- KD/KDCE/CE (T,capa)

2- Barras con test_acc,*feats(layer), raya en logit (t8), en grid ([kd,ce,kdce] [mobilenet,resnet])

3- Barras con test_acc/train_acc*feats(layer), raya en logit (t8), en grid ([kd,ce,kdce] [mobilenet,resnet])

4- majamama tiempo de convergencia, acc (epochs y tiempo)

In [ ]:
source

In [ ]:
    
alt.Chart(source).mark_circle().encode(
        alt.Y("test_acc", type='quantitative',title="Accuracy dataset natural"),#,scale=alt.Scale(zero=False,base=10,type='symlog')),
        alt.X("temp", type='quantitative',title="Accuracy dataset con ruido"),#,scale=alt.Scale(zero=False,base=10,type='symlog')),
        color=alt.Color('student', legend=alt.Legend(title="Modelo")),
    )

In [ ]:
import altair as alt
!pip install vega_datasets
from vega_datasets import data

source = data.iris()

alt.Chart(source).mark_circle().encode(
    alt.X('sepalLength', scale=alt.Scale(zero=False)),
    alt.Y('sepalWidth', scale=alt.Scale(zero=False, padding=1)),
    color='species',
    size='petalWidth'
)

### Graficos GAN
Tabla teachers, (mejores, peores, media) en KD y con feats

2- Barras con test_acc,*feats(layer), raya en logit (t8), en grid ([kd] [mobilenet,resnet])
3- Barras con test_acc/train_acc*feats(layer), raya en logit (t8), en grid ([kd] [mobilenet,resnet])
4- majamama tiempo de convergencia, acc (epochs y tiempo)

### Graficos VAE
Tabla teachers, (mejores, peores, media) en KD y con feats

1- KD/KDCE/CE (T,capa)
2- Barras con test_acc,*feats(layer), raya en logit (t8), en grid ([kd] [mobilenet,resnet])
3- Barras con test_acc/train_acc*feats(layer), raya en logit (t8), en grid ([kd] [mobilenet,resnet])
4- majamama tiempo de convergencia, acc (epochs y tiempo)

### Graficos Ruido
Tabla teachers, (mejores, peores, media) en KD y con feats

1- KD/KDCE/CE (T,capa)
2- Barras con test_acc,*feats(layer), raya en logit (t8), en grid ([kd] [mobilenet,resnet])
3- Barras con test_acc/train_acc*feats(layer), raya en logit (t8), en grid ([k] [mobilenet,resnet])
4- majamama linea de decaimiento acc (epochs y tiempo)
5- 

# 

In [84]:
import pandas as pd
import altair as alt


CE_path=["Cifar10/%s/record.json"%model for model in ["ResNet101","ResNet18","MobileNet"]]

import json
def load_teachers(CE_path):
    def flatten_record(path):
        i=json.load(open(path,'r'))
        d={}
        for k,v in i.items():
            if isinstance(v, dict) :
                for l in v.values():
                    for ki,vi in l.items():
                        #print(l.values())
                        d["/".join([k,ki])]=vi
            else:
                d[k]=v
        return d

    d={'model':[],
     'test/acc': [],
     'test/loss ': [],
     'train/acc': [],
     'train/loss ': [],
     'epoch': [],
     'train_step': [],
     'test_step': []}

    for path in CE_path:
        d['model'].append(path.split("/")[-2])
        for k,v in flatten_record(path).items():
            d[k].append(v)
    return pd.DataFrame.from_dict(d)

data={}
data['teachers']=load_teachers(CE_path)
data["students"]={'feat':{},
                  'kd':{}}


import glob
paths=glob.glob("Cifar10/**/summary.csv",recursive=True)
for path in paths:
    ks=path.split("/")[-2].split("_")
    data["students"][ks[0]][ks[1]]=pd.read_csv(path)



In [87]:
data['students']['kd']['normal']

,Unnamed: 0,test_acc,test_teacher/acc,test_loss,test_eval,train_acc,train_teacher/acc,train_loss,train_eval,epoch,...,resume,epochs,train_batch_size,test_batch_size,student,teacher,distillation,transform,dataset,exp_name
0,0,87.79,93.94,0.039678,0.419880,94.398,99.910,0.015599,0.155688,50,...,False,50,128,128,MobileNet,ResNet101,"KD,T-1","none,",None,KD_normal
1,1,89.61,93.94,0.011155,1.358676,94.688,99.898,0.007301,0.438584,50,...,False,50,128,128,MobileNet,ResNet101,"KD,T-10","none,",None,KD_normal
2,2,84.22,93.94,0.000412,2.145133,86.868,99.914,0.000267,1.368471,50,...,False,50,128,128,MobileNet,ResNet101,"KD,T-100","none,",None,KD_normal
3,3,88.62,93.94,0.022256,1.118855,94.856,99.920,0.012607,0.338454,50,...,False,50,128,128,MobileNet,ResNet101,"KD,T-5","none,",None,KD_normal
4,4,87.52,93.94,0.000992,1.762909,91.282,99.910,0.000664,0.847137,50,...,False,50,128,128,MobileNet,ResNet101,"KD,T-50","none,",None,KD_normal
5,5,89.42,93.94,0.014077,1.294886,94.856,99.894,0.009106,0.404335,50,...,False,50,128,128,MobileNet,ResNet101,"KD,T-8","none,",None,KD_normal
6,6,88.08,93.94,0.226499,0.415228,94.352,99.910,0.086092,0.156331,50,...,False,50,128,128,MobileNet,ResNet101,"KD_CE,T-1","none,",None,KD_normal
7,7,88.14,93.94,20.988221,0.419095,94.650,99.922,7.489700,0.149234,50,...,False,50,128,128,MobileNet,ResNet101,"KD_CE,T-10","none,",None,KD_normal
8,8,87.40,93.94,2283.002529,0.456600,94.552,99.916,775.190526,0.155038,50,...,False,50,128,128,MobileNet,ResNet101,"KD_CE,T-100","none,",None,KD_normal
9,9,86.95,93.94,5.555757,0.442364,94.682,99.902,1.905571,0.150378,50,...,False,50,128,128,MobileNet,ResNet101,"KD_CE,T-5","none,",None,KD_normal


In [91]:
source=data['students']['kd']['normal'].copy()
source["test_acc"]

0     87.79
1     89.61
2     84.22
3     88.62
4     87.52
5     89.42
6     88.08
7     88.14
8     87.40
9     86.95
10    87.38
11    87.69
12    90.85
13    93.09
14    88.45
15    92.65
16    91.19
17    92.91
18    92.31
19    91.68
20    91.11
21    91.54
22    91.88
23    91.55
Name: test_acc, dtype: float64

### Graficos Dataset Normal
Tabla teachers, (mejores, peores, media) en KD y con feats

1- KD/KDCE/CE (T,capa)

2- Barras con test_acc,*feats(layer), raya en logit (t8), en grid ([kd,ce,kdce] [mobilenet,resnet])

3- Barras con test_acc/train_acc*feats(layer), raya en logit (t8), en grid ([kd,ce,kdce] [mobilenet,resnet])

4- majamama tiempo de convergencia, acc (epochs y tiempo)

In [81]:
source

,sepalLength,sepalWidth,petalLength,petalWidth,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [98]:
    
alt.Chart(source).mark_circle().encode(
        alt.Y("test_acc", type='quantitative',title="Accuracy dataset natural"),#,scale=alt.Scale(zero=False,base=10,type='symlog')),
        alt.X("temp", type='quantitative',title="Accuracy dataset con ruido"),#,scale=alt.Scale(zero=False,base=10,type='symlog')),
        color=alt.Color('student', legend=alt.Legend(title="Modelo")),
    )

ValueError: student encoding field is specified without a type; the type cannot be inferred because it does not match any column in the data.

alt.Chart(...)

In [93]:
import altair as alt
!pip install vega_datasets
from vega_datasets import data

source = data.iris()

alt.Chart(source).mark_circle().encode(
    alt.X('sepalLength', scale=alt.Scale(zero=False)),
    alt.Y('sepalWidth', scale=alt.Scale(zero=False, padding=1)),
    color='species',
    size='petalWidth'
)

alt.Chart(...)

### Graficos GAN
Tabla teachers, (mejores, peores, media) en KD y con feats

2- Barras con test_acc,*feats(layer), raya en logit (t8), en grid ([kd] [mobilenet,resnet])
3- Barras con test_acc/train_acc*feats(layer), raya en logit (t8), en grid ([kd] [mobilenet,resnet])
4- majamama tiempo de convergencia, acc (epochs y tiempo)

### Graficos VAE
Tabla teachers, (mejores, peores, media) en KD y con feats

1- KD/KDCE/CE (T,capa)
2- Barras con test_acc,*feats(layer), raya en logit (t8), en grid ([kd] [mobilenet,resnet])
3- Barras con test_acc/train_acc*feats(layer), raya en logit (t8), en grid ([kd] [mobilenet,resnet])
4- majamama tiempo de convergencia, acc (epochs y tiempo)

### Graficos Ruido
Tabla teachers, (mejores, peores, media) en KD y con feats

1- KD/KDCE/CE (T,capa)
2- Barras con test_acc,*feats(layer), raya en logit (t8), en grid ([kd] [mobilenet,resnet])
3- Barras con test_acc/train_acc*feats(layer), raya en logit (t8), en grid ([k] [mobilenet,resnet])
4- majamama linea de decaimiento acc (epochs y tiempo)
5- 

# 